**ML BABA J - Creating the data set for the dashboard**

In [1]:
import pandas as pd

# Use pandas to read file
df = pd.read_excel("/Users/aya/Desktop/Welcome Aboard!/AlldataBabaJ/ship_samples_lat_lon.xlsx")

# Join all latitude values into one column
df["Latitude"] = df['lat (degrees)'].astype(float) + (df['lat (minutes)'].astype(float) / 60) + (df['lat (seconds)'].astype(float) / 3600)

# Join all longitude values into one column
df["Longitude"] = df['long (degrees)'].astype(float) + (df['long (minutes)'].astype(float) / 60) + (df['long (seconds)'].astype(float) / 3600)

print(df.head())

df_json = df.to_json()


                Source       date  lat (degrees)  lat (minutes)  \
0  BOX 1, STOLT VIKING 2022-09-30             21           25.0   
1  BOX 1, STOLT VIKING 2022-10-01             20            3.0   
2  BOX 1, STOLT VIKING 2022-10-02             18           31.0   
3  BOX 1, STOLT VIKING 2022-10-03             16           52.0   
4  BOX 1, STOLT VIKING 2022-10-04             15           19.0   

   lat (seconds)  long (degrees)  long (minutes)  long (seconds)  \
0              0             -84            36.0              48   
1             30             -80             0.0               0   
2             12             -75            12.0              24   
3             24             -70            43.0              18   
4             18             -66             9.0              30   

   flag (1 = contaminated; 2 = low pressure)   Latitude  Longitude  
0                                        NaN  21.416667 -83.386667  
1                                        NaN  20.0

In [2]:
# Replace NaN with 0 in the "flag" column and rename the column

df["flag (1 = contaminated; 2 = low pressure)"] = df["flag (1 = contaminated; 2 = low pressure)"].fillna(0)

df.rename(
    columns={
        "flag (1 = contaminated; 2 = low pressure)": "flag (0= uncontaminated, 1= contaminated, 2= low pressure)"},
    inplace=True,
)

# Use mapping to get categorical values to be descriptive and not numerical

flag_mapping = {
    0: "Uncontaminated",
    1: "Contaminated",
    2: "Low Pressure"
}
df["flag (0= uncontaminated, 1= contaminated, 2= low pressure)"] = df["flag (0= uncontaminated, 1= contaminated, 2= low pressure)"].map(flag_mapping)

# Select certain columns for a new DataFrame
correct_df = df[["Source", "date",  "Latitude", "Longitude", "flag (0= uncontaminated, 1= contaminated, 2= low pressure)"]]

print(correct_df)


                  Source       date   Latitude  Longitude  \
0    BOX 1, STOLT VIKING 2022-09-30  21.416667 -83.386667   
1    BOX 1, STOLT VIKING 2022-10-01  20.058333 -80.000000   
2    BOX 1, STOLT VIKING 2022-10-02  18.520000 -74.793333   
3    BOX 1, STOLT VIKING 2022-10-03  16.873333 -69.278333   
4    BOX 1, STOLT VIKING 2022-10-04  15.321667 -65.841667   
..                   ...        ...        ...        ...   
358          Box 21: CSL 2024-06-11  16.271667 -20.508333   
359          Box 21: CSL 2024-06-12  22.075000 -17.546667   
360          Box 21: CSL 2024-06-13  27.946667 -12.261667   
361          Box 21: CSL 2024-06-14  32.548333  -8.105000   
362          Box 21: CSL 2024-06-15  35.475000  -5.571667   

    flag (0= uncontaminated, 1= contaminated, 2= low pressure)  
0                                       Uncontaminated          
1                                       Uncontaminated          
2                                       Uncontaminated          
3      

**SCRAPE**

1) Air Quality Index 
2) CO Concentraion 
3) Weather 
4) Temperature

In [3]:
import requests
import pandas as pd
from datetime import datetime

api_key = "de26752686c975de6a1c38a998f50fec"
base_url = "http://api.openweathermap.org/data/2.5/air_pollution/history?"

# Ensure "date" is in datetime format
correct_df["date"] = pd.to_datetime(correct_df["date"])

# Convert "date" to Unix timestamps
correct_df["start"] = correct_df["date"].apply(lambda x: int(x.timestamp()))
correct_df["end"] = correct_df["start"]  # Same as start date

# Initialize the new columns in the DataFrame
correct_df["Air Quality Index (AQI)"] = None
correct_df["CO Concentration μg/m3"] = None

# Iterate through each row to call the API
for index, row in correct_df.iterrows():
    lat = row["Latitude"]
    lon = row["Longitude"]
    start = row["start"]
    end = row["end"]
    
    # Complete URL for the API call
    url = f"{base_url}lat={lat}&lon={lon}&start={start}&end={end}&appid={api_key}"
    
    # Make the API request
    response = requests.get(url)
    
    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()

        # Extract air quality index and CO component
        air_quality_index = data["list"][0]["main"]["aqi"] if data.get("list") else None
        co_component = data["list"][0]["components"]["co"] if data.get("list") else None
        
        # Print the air quality index and CO component (only to check)
        #print(f"Air Quality Index (AQI): {air_quality_index}, CO Component: {co_component}")

        # Update the specific row in the DataFrame with the new values
        correct_df.at[index, "Air Quality Index (AQI)"] = air_quality_index
        correct_df.at[index, "CO Concentration μg/m3"] = co_component

    else:
        print(f"Failed to fetch data for lat: {lat}, lon: {lon}. Status code: {response.status_code}")
        print("Response content:", response.text)  # Print the error message

# Print the updated DataFrame
print(correct_df)



/var/folders/1b/v3n5xds114d88798g8xmq0740000gn/T/ipykernel_93770/1134520085.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correct_df["date"] = pd.to_datetime(correct_df["date"])
/var/folders/1b/v3n5xds114d88798g8xmq0740000gn/T/ipykernel_93770/1134520085.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  correct_df["start"] = correct_df["date"].apply(lambda x: int(x.timestamp()))
/var/folders/1b/v3n5xds114d88798g8xmq0740000gn/T/ipykernel_93770/1134520085.py:13: SettingWithCopyWarning: 
A value is try

                  Source       date   Latitude  Longitude  \
0    BOX 1, STOLT VIKING 2022-09-30  21.416667 -83.386667   
1    BOX 1, STOLT VIKING 2022-10-01  20.058333 -80.000000   
2    BOX 1, STOLT VIKING 2022-10-02  18.520000 -74.793333   
3    BOX 1, STOLT VIKING 2022-10-03  16.873333 -69.278333   
4    BOX 1, STOLT VIKING 2022-10-04  15.321667 -65.841667   
..                   ...        ...        ...        ...   
358          Box 21: CSL 2024-06-11  16.271667 -20.508333   
359          Box 21: CSL 2024-06-12  22.075000 -17.546667   
360          Box 21: CSL 2024-06-13  27.946667 -12.261667   
361          Box 21: CSL 2024-06-14  32.548333  -8.105000   
362          Box 21: CSL 2024-06-15  35.475000  -5.571667   

    flag (0= uncontaminated, 1= contaminated, 2= low pressure)       start  \
0                                       Uncontaminated          1664496000   
1                                       Uncontaminated          1664582400   
2                                

In [4]:
# Downloaded historical data from free api called virtualcrossing
# we downloaded the locations from the corrected_df and used it as is to find the weather
# now we will add the relevant columns to the df

# read the csv file

import pandas as pd

# Read the data into a DataFrame (make sure the file path is correct)
df = pd.read_csv('historicaldata.csv', sep=";")

# Filter the DataFrame to only include Latitude, Longitude, Temp, and Conditions
df_filtered = df[['Latitude', 'Longitude', 'temp', 'conditions']]
df_filtered["tempC"] = ((df_filtered['temp'] - 32) * 5/9).round(2)

correct_df["Weather"] = df_filtered["conditions"]
correct_df["Temperature C"] = df_filtered["tempC"]

print(correct_df.head(100))

#FINALLY save AS JSON for website


correct_df.to_json("/Users/aya/Desktop/Welcome Aboard!/static/data.json", orient='records')

                 Source       date   Latitude  Longitude  \
0   BOX 1, STOLT VIKING 2022-09-30  21.416667 -83.386667   
1   BOX 1, STOLT VIKING 2022-10-01  20.058333 -80.000000   
2   BOX 1, STOLT VIKING 2022-10-02  18.520000 -74.793333   
3   BOX 1, STOLT VIKING 2022-10-03  16.873333 -69.278333   
4   BOX 1, STOLT VIKING 2022-10-04  15.321667 -65.841667   
..                  ...        ...        ...        ...   
95          BOX 11: CSA 2023-05-03 -15.690000 -35.916667   
96           BOX 7, CSL 2023-05-03  23.758333 -16.523333   
97          BOX 11: CSA 2023-05-04  -8.156667 -32.906667   
98           BOX 7, CSL 2023-05-04  17.435000 -19.446667   
99          BOX 17: CSA 2023-05-05  -3.695000 -29.426667   

   flag (0= uncontaminated, 1= contaminated, 2= low pressure)       start  \
0                                      Uncontaminated          1664496000   
1                                      Uncontaminated          1664582400   
2                                      Uncontami

/var/folders/1b/v3n5xds114d88798g8xmq0740000gn/T/ipykernel_93770/505912584.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered["tempC"] = ((df_filtered['temp'] - 32) * 5/9).round(2)


In [10]:
import requests

# API keys
API_KEY_WEATHER = "de26752686c975de6a1c38a998f50fec"
API_KEY_AIR_POLLUTION = "de26752686c975de6a1c38a998f50fec"
BASE_URL_WEATHER = "https://api.openweathermap.org/data/2.5/weather?"
BASE_URL_AIR_POLLUTION = "http://api.openweathermap.org/data/2.5/air_pollution?"

# Function to get current weather data
def get_current_weather(lat, lon):
    url = f"{BASE_URL_WEATHER}lat={lat}&lon={lon}&appid={API_KEY_WEATHER}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            if "weather" in data and len(data["weather"]) > 0:
                weather = {
                    "main": data["weather"][0]["main"],  # Weather condition (Rain, Clear, etc.)
                    "temp": data["main"]["temp"],  # Current temperature in °C
                }
                return weather
            else:
                print("Weather data is missing or in an unexpected format.")
                return None
        except (KeyError, IndexError) as e:
            print(f"Error parsing weather data: {e}")
            return None
    else:
        print(f"Failed to fetch weather data. Status code: {response.status_code}")
        return None

# Function to get air pollution data
def get_air_pollution(lat, lon):
    url = f"{BASE_URL_AIR_POLLUTION}lat={lat}&lon={lon}&appid={API_KEY_AIR_POLLUTION}"
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            if "list" in data:
                air_quality = data["list"][0]["main"]["aqi"]
                co = data["list"][0]["components"]["co"]
                return {
                    "aqi": air_quality,
                    "co": co
                }
            else:
                return None
        except KeyError as e:
            print(f"Error parsing air pollution data: {e}")
            return None
    else:
        print(f"Failed to fetch air pollution data. Status code: {response.status_code}")
        return None

# Example latitude and longitude
lat = 40.7128  # Example latitude (New York City)
lon = -74.0060  # Example longitude (New York City)

# Get weather and air pollution data
weather = get_current_weather(lat, lon)
air_pollution_data = get_air_pollution(lat, lon)

# Print the data
print("Weather Data:", weather)  # Print weather data
print("Air Pollution Data:", air_pollution_data)  # Print air pollution data


Weather Data: {'main': 'Clear', 'temp': -4.37}
Air Pollution Data: {'aqi': 2, 'co': 280.38}
